In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

2023-11-22 13:21:21.828329: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
!nvidia-smi

Wed Nov 22 13:21:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:37:00.0 Off |                    0 |
| N/A   27C    P8     6W /  75W |      0MiB /  7680MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
from tensorflow.keras import backend
import _pickle as cPickle

In [4]:
def load_batch(fpath, label_key="labels"):
    """Internal utility for parsing CIFAR data.

    Args:
        fpath: path the file to parse.
        label_key: key for label data in the retrieve
            dictionary.

    Returns:
        A tuple `(data, labels)`.
    """
    with open(fpath, "rb") as f:
        d = cPickle.load(f, encoding="bytes")
        # decode utf8
        d_decoded = {}
        for k, v in d.items():
            d_decoded[k.decode("utf8")] = v
        d = d_decoded
    data = d["data"]
    labels = d[label_key]

    data = data.reshape(data.shape[0], 3, 32, 32)
    return data, labels

In [5]:
def load_data(label_mode="fine"):
    path = './cifar-100-python'

    fpath = os.path.join(path, "train")
    x_train, y_train = load_batch(fpath, label_key=label_mode + "_labels")

    fpath = os.path.join(path, "test")
    x_test, y_test = load_batch(fpath, label_key=label_mode + "_labels")

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    if backend.image_data_format() == "channels_last":
        x_train = x_train.transpose(0, 2, 3, 1)
        x_test = x_test.transpose(0, 2, 3, 1)

    return (x_train, y_train), (x_test, y_test)

In [6]:
(x_train_val, y_train_val), (x_test, y_test) = load_data()

In [7]:
x_train_val.shape, y_train_val.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [8]:
val_size = 0.2
random_seed = 0
num_train = x_train_val.shape[0]
indices = list(range(num_train))
split = int(np.floor(val_size * num_train))
np.random.seed(random_seed)
np.random.shuffle(indices)
train_idx, val_idx = indices[split:], indices[:split]

In [9]:
x_train, x_val = x_train_val[train_idx], x_train_val[val_idx]
y_train, y_val = y_train_val[train_idx], y_train_val[val_idx] 
x_train.shape, x_val.shape

((40000, 32, 32, 3), (10000, 32, 32, 3))

In [10]:
import os
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_source_inputs
from tensorflow.keras.layers import Activation, Add, Concatenate, Conv2D, GlobalMaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dense
from tensorflow.keras.layers import MaxPool2D, AveragePooling2D, BatchNormalization, Lambda, DepthwiseConv2D
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

In [11]:
def channel_split(x, name=''):
    # equipartition
    in_channles = x.shape.as_list()[-1]
    ip = in_channles // 2
    c_hat = Lambda(lambda z: z[:, :, :, 0:ip], name='%s/sp%d_slice' % (name, 0))(x)
    c = Lambda(lambda z: z[:, :, :, ip:], name='%s/sp%d_slice' % (name, 1))(x)
    return c_hat, c

def channel_shuffle(x):
    height, width, channels = x.shape.as_list()[1:]
    channels_per_split = channels // 2
    x = K.reshape(x, (-1, height, width, 2, channels_per_split))
    x = K.permute_dimensions(x, (0,1,2,4,3))
    x = K.reshape(x, (-1, height, width, channels))
    return x


def shuffle_unit(inputs, out_channels, bottleneck_ratio,strides=2,stage=1,block=1):
    if K.image_data_format() == 'channels_last':
        bn_axis = -1
    else:
        raise ValueError('Only channels last supported')

    prefix = 'stage{}/block{}'.format(stage, block)
    bottleneck_channels = int(out_channels * bottleneck_ratio)
    if strides < 2:
        c_hat, c = channel_split(inputs, '{}/spl'.format(prefix))
        inputs = c

    x = Conv2D(bottleneck_channels, kernel_size=(1,1), strides=1, padding='same', name='{}/1x1conv_1'.format(prefix))(inputs)
    x = BatchNormalization(axis=bn_axis, name='{}/bn_1x1conv_1'.format(prefix))(x)
    x = Activation('relu', name='{}/relu_1x1conv_1'.format(prefix))(x)
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same', name='{}/3x3dwconv'.format(prefix))(x)
    x = BatchNormalization(axis=bn_axis, name='{}/bn_3x3dwconv'.format(prefix))(x)
    x = Conv2D(bottleneck_channels, kernel_size=1,strides=1,padding='same', name='{}/1x1conv_2'.format(prefix))(x)
    x = BatchNormalization(axis=bn_axis, name='{}/bn_1x1conv_2'.format(prefix))(x)
    x = Activation('relu', name='{}/relu_1x1conv_2'.format(prefix))(x)

    if strides < 2:
        ret = Concatenate(axis=bn_axis, name='{}/concat_1'.format(prefix))([x, c_hat])
    else:
        s2 = DepthwiseConv2D(kernel_size=3, strides=2, padding='same', name='{}/3x3dwconv_2'.format(prefix))(inputs)
        s2 = BatchNormalization(axis=bn_axis, name='{}/bn_3x3dwconv_2'.format(prefix))(s2)
        s2 = Conv2D(bottleneck_channels, kernel_size=1,strides=1,padding='same', name='{}/1x1_conv_3'.format(prefix))(s2)
        s2 = BatchNormalization(axis=bn_axis, name='{}/bn_1x1conv_3'.format(prefix))(s2)
        s2 = Activation('relu', name='{}/relu_1x1conv_3'.format(prefix))(s2)
        ret = Concatenate(axis=bn_axis, name='{}/concat_2'.format(prefix))([x, s2])

    ret = Lambda(channel_shuffle, name='{}/channel_shuffle'.format(prefix))(ret)

    return ret


def block(x, channel_map, bottleneck_ratio, repeat=1, stage=1):
    x = shuffle_unit(x, out_channels=channel_map[stage-1],
                      strides=2,bottleneck_ratio=bottleneck_ratio,stage=stage,block=1)

    for i in range(1, repeat+1):
        x = shuffle_unit(x, out_channels=channel_map[stage-1],strides=1,
                          bottleneck_ratio=bottleneck_ratio,stage=stage, block=(1+i))

    return x

In [12]:
def create_augmentation_layer():
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.1),
    ])
    return data_augmentation

In [13]:
def create_preprocess_layer():
    data_preprocessing = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Resizing(224, 224),
        tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    ])
    return data_preprocessing

In [14]:
def ShuffleNetV2(include_top=True,
                 input_tensor=None,
                 scale_factor=1.0,
                 pooling='max',
                 input_shape=(224,224,3),
                 load_model=None,
                 num_shuffle_units=[3,7,3],
                 bottleneck_ratio=1,
                 classes=1000,
                 augment=False,
                 dropout=False,
                 L1=False,
                 L2=False):
    if K.backend() != 'tensorflow':
        raise RuntimeError('Only tensorflow supported for now')
    name = 'ShuffleNetV2_{}_{}_{}'.format(scale_factor, bottleneck_ratio, "".join([str(x) for x in num_shuffle_units]))
    out_dim_stage_two = {0.5:48, 1:116, 1.5:176, 2:244}

    if pooling not in ['max', 'avg']:
        raise ValueError('Invalid value for pooling')
    if not (float(scale_factor)*4).is_integer():
        raise ValueError('Invalid value for scale_factor, should be x over 4')
    exp = np.insert(np.arange(len(num_shuffle_units), dtype=np.float32), 0, 0)  # [0., 0., 1., 2.]
    out_channels_in_stage = 2**exp
    out_channels_in_stage *= out_dim_stage_two[bottleneck_ratio]  #  calculate output channels for each stage
    out_channels_in_stage[0] = 24  # first stage has always 24 output channels
    out_channels_in_stage *= scale_factor
    out_channels_in_stage = out_channels_in_stage.astype(int)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    
    # data preprocessing
    x = create_preprocess_layer()(img_input)
    
    # data augmentation
    if augment:
        x = create_augmentation_layer()(x)

    # create shufflenet architecture
    x = Conv2D(filters=out_channels_in_stage[0], kernel_size=(3, 3), padding='same', use_bias=False, strides=(2, 2),
               activation='relu', name='conv1')(x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='maxpool1')(x)

    # create stages containing shufflenet units beginning at stage 2
    for stage in range(len(num_shuffle_units)):
        repeat = num_shuffle_units[stage]
        x = block(x, out_channels_in_stage,
                   repeat=repeat,
                   bottleneck_ratio=bottleneck_ratio,
                   stage=stage + 2)

    if bottleneck_ratio < 2:
        k = 1024
    else:
        k = 2048
    x = Conv2D(k, kernel_size=1, padding='same', strides=1, name='1x1conv5_out', activation='relu')(x)

    if pooling == 'avg':
        x = GlobalAveragePooling2D(name='global_avg_pool')(x)
    elif pooling == 'max':
        x = GlobalMaxPooling2D(name='global_max_pool')(x)
        
    if dropout:
        x = layers.Dropout(0.5)(x)

    if include_top:
        if L1:
            x = Dense(classes, name='fc', kernel_regularizer=tf.keras.regularizers.l1(0.001))(x)
        elif L2:
            x = Dense(classes, name='fc', kernel_regularizer=tf.keras.regularizers.l2(0.0001))(x)
        else:
            x = Dense(classes, name='fc')(x)
        x = Activation('softmax', name='softmax')(x)

    if input_tensor:
        inputs = get_source_inputs(input_tensor)

    else:
        inputs = img_input

    model = Model(inputs, x, name=name)

    if load_model:
        model.load_weights('', by_name=True)

    return model

In [15]:
model = ShuffleNetV2(include_top=True, input_shape=(32, 32, 3), bottleneck_ratio=1, classes=100, L1=True)
model.summary()

2023-11-22 08:43:29.450810: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-11-22 08:43:29.642934: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:12:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.56GiB deviceMemoryBandwidth: 298.08GiB/s
2023-11-22 08:43:29.642977: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-11-22 08:43:29.792712: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-11-22 08:43:29.835510: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-11-22 08:43:29.854852: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so.10
2023-11-

Model: "ShuffleNetV2_1.0_1_373"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 24) 648         sequential[0][0]                 
__________________________________________________________________________________________________
maxpool1 (MaxPooling2D)         (None, 56, 56, 24)   0           conv1[0][0]                      
_____________________________________________________________________________

In [16]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [8]:
exp_name = './L1'
os.makedirs(exp_name, exist_ok=True)

In [18]:
batch_size = 64
loss = 'sparse_categorical_crossentropy'
num_epochs = 100
optim = "adam"
mcp_save = ModelCheckpoint(f'{exp_name}/shufflenetv2_model.hdf5', save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
rlronp = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=0.000001)

model.compile(
    loss=loss,
    optimizer=optim,
    metrics=['sparse_categorical_accuracy']
)

In [19]:
hist = model.fit(
    x=x_train,y=y_train,
    epochs=num_epochs,
    batch_size=batch_size,
    validation_data=(x_val,y_val),
    verbose=1,
    callbacks=[mcp_save,rlronp],
    shuffle=True
)

train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['sparse_categorical_accuracy']
val_acc=hist.history['val_sparse_categorical_accuracy']
np.savez(os.path.join(exp_name, 'fit_history.npz'), train_loss=train_loss, val_loss=val_loss, train_acc=train_acc, val_acc=val_acc)

Epoch 1/100


2023-11-22 08:43:53.039577: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-11-22 08:43:54.031173: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


625/625 [==============================] - 137s 219ms/step - loss: 5.3477 - sparse_categorical_accuracy: 0.0980 - val_loss: 4.9336 - val_sparse_categorical_accuracy: 0.0329
Epoch 2/100
625/625 [==============================] - 140s 225ms/step - loss: 3.4893 - sparse_categorical_accuracy: 0.2209 - val_loss: 3.5797 - val_sparse_categorical_accuracy: 0.2160
Epoch 3/100
625/625 [==============================] - 141s 225ms/step - loss: 3.0292 - sparse_categorical_accuracy: 0.3069 - val_loss: 3.0697 - val_sparse_categorical_accuracy: 0.3009
Epoch 4/100
625/625 [==============================] - 140s 224ms/step - loss: 2.6421 - sparse_categorical_accuracy: 0.3861 - val_loss: 3.0987 - val_sparse_categorical_accuracy: 0.3237
Epoch 5/100
625/625 [==============================] - 140s 225ms/step - loss: 2.3327 - sparse_categorical_accuracy: 0.4549 - val_loss: 2.5255 - val_sparse_categorical_accuracy: 0.4107
Epoch 6/100
625/625 [==============================] - 141s 225ms/step - loss: 2.0796 -

Epoch 34/100
625/625 [==============================] - 140s 224ms/step - loss: 0.1026 - sparse_categorical_accuracy: 0.9998 - val_loss: 2.2696 - val_sparse_categorical_accuracy: 0.5369
Epoch 35/100
625/625 [==============================] - 140s 224ms/step - loss: 0.1025 - sparse_categorical_accuracy: 0.9998 - val_loss: 2.2697 - val_sparse_categorical_accuracy: 0.5367
Epoch 36/100
625/625 [==============================] - 140s 224ms/step - loss: 0.1022 - sparse_categorical_accuracy: 0.9999 - val_loss: 2.2710 - val_sparse_categorical_accuracy: 0.5361
Epoch 37/100
625/625 [==============================] - 140s 224ms/step - loss: 0.1022 - sparse_categorical_accuracy: 0.9998 - val_loss: 2.2715 - val_sparse_categorical_accuracy: 0.5351
Epoch 38/100
625/625 [==============================] - 140s 224ms/step - loss: 0.1019 - sparse_categorical_accuracy: 0.9999 - val_loss: 2.2721 - val_sparse_categorical_accuracy: 0.5352
Epoch 39/100
625/625 [==============================] - 140s 224ms/ste

625/625 [==============================] - 140s 224ms/step - loss: 0.0960 - sparse_categorical_accuracy: 0.9998 - val_loss: 2.2982 - val_sparse_categorical_accuracy: 0.5317
Epoch 79/100
625/625 [==============================] - 140s 224ms/step - loss: 0.0959 - sparse_categorical_accuracy: 0.9999 - val_loss: 2.2985 - val_sparse_categorical_accuracy: 0.5316
Epoch 80/100
625/625 [==============================] - 140s 225ms/step - loss: 0.0957 - sparse_categorical_accuracy: 0.9999 - val_loss: 2.3002 - val_sparse_categorical_accuracy: 0.5312
Epoch 81/100
625/625 [==============================] - 140s 224ms/step - loss: 0.0955 - sparse_categorical_accuracy: 0.9999 - val_loss: 2.2993 - val_sparse_categorical_accuracy: 0.5316
Epoch 82/100
625/625 [==============================] - 140s 224ms/step - loss: 0.0954 - sparse_categorical_accuracy: 0.9998 - val_loss: 2.3005 - val_sparse_categorical_accuracy: 0.5317
Epoch 83/100
625/625 [==============================] - 140s 224ms/step - loss: 0.0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
train_loss=model.history.history['loss']
val_loss=model.history.history['val_loss']
train_acc=model.history.history['sparse_categorical_accuracy']
val_acc=model.history.history['val_sparse_categorical_accuracy']
np.savez(os.path.join(exp_name, 'fit_history.npz'), train_loss=train_loss, val_loss=val_loss, train_acc=train_acc, val_acc=val_acc)

In [9]:
hdf5_model = f'{exp_name}/shufflenetv2_model.hdf5'
loaded_model_from_hdf5 = tf.keras.models.load_model(hdf5_model)
loaded_model_from_hdf5.evaluate(x_test,y_test,batch_size=1)

2023-11-22 13:22:02.998669: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-11-22 13:22:03.018381: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:37:00.0 name: Tesla P4 computeCapability: 6.1
coreClock: 1.1135GHz coreCount: 20 deviceMemorySize: 7.43GiB deviceMemoryBandwidth: 178.99GiB/s
2023-11-22 13:22:03.018422: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-11-22 13:22:03.151450: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-11-22 13:22:03.183799: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-11-22 13:22:03.202911: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so.10
2023-11

10000/10000 [==============================] - 52s 5ms/step - loss: 2.1171 - sparse_categorical_accuracy: 0.5556


[2.117079734802246, 0.5555999875068665]